In [2]:
#加载飞桨、Numpy和相关类库
import numpy as np
import pyeeg as pe
import os
import random
from PIL import Image

In [ ]:
def data_set_partition(
    dir='/home/aistudio/data/data57707/1280x258050(32x8064+2).npy',
    trainNum=960
    ):
    """
    数据集划分，随机打乱数据，划分后保存为训练集数据文件和测试集数据文件
    dir         数据集路径
    trainNum    训练样本数
    """
    data=np.load(dir)
    print('32通道，AV维度的数据shape：',data.shape)
    #np.random.seed(0)
    np.random.shuffle(data)

    trainData=data[:trainNum]
    testData=data[trainNum:]
    np.save('work/data/trainData.npy',trainData)
    np.save('work/data/testData.npy',testData)
    print('数据集划分成功！')
    print('trainData.shape=',trainData.shape)
    print('testData.shape=',testData.shape)

#%time data_set_partition()

In [ ]:
def data_reshape_feature_extraction(inputDir,outputDir):
    """加载数据集，塑造数据集形状，特征提取并保存（未去基线）"""
    data=np.load(inputDir)
    features=data[:,:-2]
    labels=data[:,-2:]
    features=features.reshape((data.shape[0],32,-1))
    print(inputDir,':')
    print('features.shape=',features.shape)
    print('labels.shape=',labels.shape)

    channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
    band = [4,8,12,16,25,45] #5 bands
    window_size = 256 #Averaging band power of 2 sec
    step_size = 16 #Each 0.125 sec update once
    sample_rate = 128 #Sampling rate of 128 Hz

    newData=[]
    for i in range(features.shape[0]):    #样本数
        single_channel_features=[]
        #for j in channel:
        for j in range(32):
            X = features[i][j]    #Slice raw data over 2 sec, at interval of 0.125 sec
            Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
            single_channel_features += list(Y[0])
        single_channel_features += list(labels[i])    #添加标签
        newData.append(single_channel_features)
    
    data=np.array(newData)
    print('data.shape=',data.shape)
    np.save(outputDir,data)

In [ ]:
# """特征提取"""
# trainDataDir='work/data/trainData.npy'
# testDataDir='work/data/testData.npy'
# trainFftDir='work/data/trainFFT.npy'
# testFftDir='work/data/testFFT.npy'
# data_reshape_feature_extraction(trainDataDir,trainFftDir)
# data_reshape_feature_extraction(testDataDir,testFftDir)

In [ ]:
def Gaussian_normalization():
    """高斯归一化"""

    #训练样本归一化
    data=np.load('work/data/trainFFT.npy')
    feature=data[:,:-2]
    mean=feature.mean(axis=0)
    std=feature.std(axis=0)
    feature-=mean
    feature/=std

    np.save('work/data/trainNormalization.npy',data)

    #测试样本归一化
    data=np.load('work/data/testFFT.npy')
    feature=data[:,:-2]
    feature-=mean
    feature/=std

    np.save('work/data/testNormalization.npy',data)

#%time Gaussian_normalization()